In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd

from pathlib import Path
# from pandas_profiling import ProfileReport
data_dir = Path("./data")


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        pass
        # print(os.path.join(dirname, filename))

# This is a new line

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<div style="background-color: #f0f0f0; padding: 10px; border: 2px solid #4a4a4a; border-radius: 5px">
<h2 style="color: #3b3b3b">Available data 🗓📌</h2>

<h3 style="color: #3b3b3b">What does the data include?</h3>

- Two measures of learning engagement  (`pct_access` and `engagement_index`) aggregated over **234** US school districts.
- The top **372** ed-tech products (out of **10000**) identified by the Chrome browser extension [learnplatform](https://learnplatform.com/).
- Information about the **234** school districts - heavily anonymized with sideeffects of introduced ranges (instead of actual values) and a lot of missing values.

<h2 style="color: #3b3b3b">Understanding the engagement data 🤓📚</h2>

<h3 style="color: #3b3b3b">Given definitions</h3> 

- `pct_access`: Percentage of students in the district have at least one page-load event of a given product and on a given day

    - **Example**: 15% of students in a district in Utah engaged at least once with  on Monday April 14.
    - **Question**: Some numbers are > 1. What should that mean?

- `engagement_index`: Total page-load events per one thousand students of a given product and on a given day

    - **In simple terms**: Shows how active the students are in the district.
    - **Example**: 341 engagement events per 1000 students were recorded in Chicago on Monday April 14.

<h2 style="color: #3b3b3b">Understanding the product data 🤓📚</h2>

<h3 style="color: #3b3b3b">Sector(s)</h3>

- **Issues**: 
    - What does the sector `PreK-12; Higher Ed; Corporate` exactly cover?
    - `LC - Study Tools` contains Grammarly, `Other` contains `Facebook` -- check relevance for each hypothesis!
</div>

<div style="background-color: #f0f0f0; padding: 10px; border: 2px solid #4a4a4a; border-radius: 5px">
<h2 style="color: #3b3b3b">Exploration Ideas 💡</h2>

1. Did school topics change pre/post Covid? 
    - **Hypothesis**: Pre-covid most online education was math/science. Post-covid other areas got significantly more traction.
2. Does race or economical background of a district play a role in online learning adoption?
3. Analysing the primary essential functions of learning products:
    - Are there significant differences in engagement when comparing the categories `Learning & Curriculum (LC)`, `CM = Classroom Management` and `SDO = School & District Operations`?
4. Do `pptotalraw` of a district and online education engagement correlate?
5. Did "real" online learning increase or just usage of "other" tools like e.g. Google Docs etc.?
6. Do increased online engagement rates of online learning platforms correlate with e.g. increase in social media usage?
    - What is the ratio between distractive vs. actual educational tools?
    - More general question: Which role do tools which are commonly known for being distracting (e.g. Facebook) play in online education?
    - **TODO**: Compare page loads of e.g. `Google Docs` and learning platforms.

</div>

In [4]:
import glob
import pandas as pd 

all_file_names = list(data_dir.glob("engagement_data/*.csv"))
data_of_district = []

for filename in all_file_names:
    df = pd.read_csv(filename, index_col=None, header=0)
    district_id = filename.stem
    df["district_id"] = district_id
    data_of_district.append(df)

engagement = pd.concat(data_of_district)
# del data_of_district
engagement = engagement.reset_index(drop=True)
engagement.head()

,time,lp_id,pct_access,engagement_index,district_id
0,2020-01-01,29322.0,0.04,1.30,3188
1,2020-01-01,57084.0,0.04,0.43,3188
2,2020-01-01,99916.0,0.09,5.20,3188
3,2020-01-01,58235.0,0.00,NaN,3188
4,2020-01-01,11585.0,0.04,1.30,3188


In [5]:

products = pd.read_csv(data_dir / "products_info.csv")

products['Basic_category'] = 'x'
for i in range(len(products)):
    if pd.isna(products['Primary Essential Function'][i]) == False:
        products['Basic_category'][i] = products['Primary Essential Function'][i].split('-')[0][:-1]

products.head()

/var/folders/zy/jpm2phcn43d1_y82b5q3n6280000gq/T/ipykernel_4743/824714450.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products['Basic_category'][i] = products['Primary Essential Function'][i].split('-')[0][:-1]


,LP ID,URL,Product Name,Provider/Company Name,Sector(s),Primary Essential Function,Basic_category
0,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC - Digital Learning Platforms,LC
1,66933,https://abcmouse.com,ABCmouse.com,"Age of Learning, Inc",PreK-12,LC - Digital Learning Platforms,LC
2,50479,https://www.abcya.com,ABCya!,"ABCya.com, LLC",PreK-12,"LC - Sites, Resources & Reference - Games & Si...",LC
3,92993,http://www.aleks.com/,ALEKS,McGraw-Hill PreK-12,PreK-12; Higher Ed,LC - Digital Learning Platforms,LC
4,73104,https://www.achieve3000.com/,Achieve3000,Achieve3000,PreK-12,LC - Digital Learning Platforms,LC


In [6]:


districts = pd.read_csv(data_dir /"districts_info.csv")
districts.dropna(inplace = True)
districts.head()

,district_id,state,locale,pct_black/hispanic,pct_free/reduced,county_connections_ratio,pp_total_raw
0,8815,Illinois,Suburb,"[0, 0.2[","[0, 0.2[","[0.18, 1[","[14000, 16000["
2,4921,Utah,Suburb,"[0, 0.2[","[0.2, 0.4[","[0.18, 1[","[6000, 8000["
5,5987,Wisconsin,Suburb,"[0, 0.2[","[0, 0.2[","[0.18, 1[","[10000, 12000["
6,3710,Utah,Suburb,"[0, 0.2[","[0.4, 0.6[","[0.18, 1[","[6000, 8000["
7,7177,North Carolina,Suburb,"[0.2, 0.4[","[0.2, 0.4[","[0.18, 1[","[8000, 10000["


## Average anonymized data ranges

Data was anonymized.

In [7]:
from statistics import mean

for col in ['pct_black/hispanic', 'pct_free/reduced', 'pp_total_raw', 'county_connections_ratio']:
  districts[col] = districts[col].apply(lambda val: mean([float(x) for x in val[1:-1].split(',')]))

districts.head()

,district_id,state,locale,pct_black/hispanic,pct_free/reduced,county_connections_ratio,pp_total_raw
0,8815,Illinois,Suburb,0.1,0.1,0.59,15000.0
2,4921,Utah,Suburb,0.1,0.3,0.59,7000.0
5,5987,Wisconsin,Suburb,0.1,0.1,0.59,11000.0
6,3710,Utah,Suburb,0.1,0.5,0.59,7000.0
7,7177,North Carolina,Suburb,0.3,0.3,0.59,9000.0


In [8]:
#---- DASK -----#
# import dask.dataframe as dd

# # engagement_data_sample = pd.read_csv(data_dir / "engagement_data/1000.csv")

# all_file_names = list(data_dir.glob("engagement_data/*.csv"))

# data_of_district = []

# for filename in all_file_names:
#     # df = pd.read_csv(filename, index_col=None, header=0)
#     df = dd.read_csv(filename, header=0, assume_missing=True)
#     district_id = filename.stem
#     df["district_id"] = district_id
#     data_of_district.append(df)
    
# engagement = dd.multi.concat(data_of_district)
# # del data_of_district
# engagement = engagement.reset_index(drop=True)
# engagement.head()

# products = dd.read_csv(data_dir / "products_info.csv")
# print(engagement['lp_id'].nunique().compute())
# print(products['LP ID'].nunique().compute())

# districts = dd.read_csv(data_dir /"districts_info.csv")
# districts.head()

In [9]:
print(engagement['lp_id'].nunique())
print(products['LP ID'].nunique())

merged_data = pd.merge(products, engagement, left_on = 'LP ID', right_on = 'lp_id')
print('merged_data', merged_data)
merged_data['district_id'] = merged_data['district_id'].astype('int64')
merged_data = pd.merge(merged_data, districts, on = 'district_id')
print('merged_data', merged_data)


8646
372
merged_data           LP ID                         URL                  Product Name  \
0         13117  https://www.splashmath.com                   SplashLearn   
1         13117  https://www.splashmath.com                   SplashLearn   
2         13117  https://www.splashmath.com                   SplashLearn   
3         13117  https://www.splashmath.com                   SplashLearn   
4         13117  https://www.splashmath.com                   SplashLearn   
...         ...                         ...                           ...   
11691582  87841         http://edpuzzle.com  Edpuzzle - Free (Basic Plan)   
11691583  87841         http://edpuzzle.com  Edpuzzle - Free (Basic Plan)   
11691584  87841         http://edpuzzle.com  Edpuzzle - Free (Basic Plan)   
11691585  87841         http://edpuzzle.com  Edpuzzle - Free (Basic Plan)   
11691586  87841         http://edpuzzle.com  Edpuzzle - Free (Basic Plan)   

         Provider/Company Name Sector(s)       Primary

In [10]:
merged_data.drop(['URL', 'lp_id'], axis = 1, inplace = True)
merged_data.columns

Index(['LP ID', 'Product Name', 'Provider/Company Name', 'Sector(s)',
       'Primary Essential Function', 'Basic_category', 'time', 'pct_access',
       'engagement_index', 'district_id', 'state', 'locale',
       'pct_black/hispanic', 'pct_free/reduced', 'county_connections_ratio',
       'pp_total_raw'],
      dtype='object')

In [52]:


st_acсess = merged_data.groupby(['state', 'time']).agg({'pct_access': 'mean'}).reset_index()
st_eng = merged_data.groupby(['state', 'time']).agg({'engagement_index': 'mean'}).reset_index()
loc_acсess = merged_data.groupby(['locale', 'time']).agg({'pct_access': 'mean'}).reset_index()
loc_eng = merged_data.groupby(['locale', 'time']).agg({'engagement_index': 'mean'}).reset_index()
cat_acсess = merged_data.groupby(['Basic_category', 'time']).agg({'pct_access': 'mean'}).reset_index()
cat_eng = merged_data.groupby(['Basic_category', 'time']).agg({'engagement_index': 'mean'}).reset_index()

# for i in [st_acсess, st_eng, loc_acсess, loc_eng, cat_acсess, cat_eng]:
#     i['day_of_week'] = i['time'].dt.dayofweek
    



In [53]:


def create_running_average(df: pd.DataFrame, category: str, kernel_size: int=8):
    runn_avg_colname = f"running_average_{category}"
    df[runn_avg_colname] = 0
    for state in df.state:
        state_mask = df.state == state
        column = df.loc[state_mask, category].values
        running_mean = np.convolve(column, np.ones(kernel_size)/kernel_size, mode="same")
        df.loc[state_mask, runn_avg_colname] = running_mean

In [56]:
st_acсess.head(3)
create_running_average(st_acсess, 'pct_access')
create_running_average(st_eng, 'engagement_index')

In [25]:
!pip install plotly

In [15]:
import plotly as py
import plotly.express as px

In [20]:
st_acсess

,state,time,pct_access
0,Florida,2020-01-01,0.024938
1,Florida,2020-01-02,0.025029
2,Florida,2020-01-03,0.028268
3,Florida,2020-01-04,0.027079
4,Florida,2020-01-05,0.026062
...,...,...,...
4845,Wisconsin,2020-12-27,0.179570
4846,Wisconsin,2020-12-28,0.200551
4847,Wisconsin,2020-12-29,0.172923
4848,Wisconsin,2020-12-30,0.190388


In [58]:
fig = px.line(st_eng, x="time", y="running_average_engagement_index", color="state", line_group="state")

fig.update_layout(plot_bgcolor = 'white', title = 'Dynamics of pct_access of all products by states', 
                  title_font_family = 'monospace', title_font_color = '#221f1f', title_font_size = 20, title_x = 0.5)
fig.update_xaxes(showline = True, linecolor = '#f5f2f2', linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)
fig.update_yaxes(showline = True, linecolor = '#f5f2f2', 
                 showgrid = True, gridwidth = 1, gridcolor = '#f5f2f2',
                 linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)

fig.add_vline(x = '2020-03-11', line_width = 3, line_color="red")

fig.add_annotation(
        x='2020-03-11',
        y=2.7,
        text="WHO has declared Covid-19 a pandemic",
        showarrow=True,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            ),
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax= 130,
        ay=1
        )

fig.add_vrect(x0="2020-06-01", x1="2020-08-31", fillcolor="yellow", opacity=0.25, line_width=0)

fig.add_annotation(
        x='2020-07-15',
        y=2.25,
        text="Summer holidays",
        showarrow=False,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            )
        )

fig.update_traces(line_width=1)

fig.show()